MULTINOMIAL 2 LABEL STEM COUNTVECTORIZER NLTK

In [ ]:
import pandas as pd
import nltk
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
from nltk.stem import SnowballStemmer
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.model_selection import train_test_split
from sklearn.naive_bayes import MultinomialNB
from sklearn.metrics import classification_report
from google.colab import drive
import joblib

# Montar Google Drive
drive.mount('/content/drive')

nltk.download('punkt')
nltk.download('stopwords')

Mounted at /content/drive


[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


True

In [ ]:
# Inicializar el stemmer en inglés
stemmer = SnowballStemmer('english')

# Función para realizar stemming en el texto
def stem_text(text):
    # Tokenizar el texto
    tokens = word_tokenize(text)
    # Realizar stemming en cada token y excluir las stopwords
    stemmed_words = [stemmer.stem(word) for word in tokens if word.lower() not in stopwords.words('english')]
    return ' '.join(stemmed_words)


In [ ]:
# Cargar los datos
data = pd.read_csv('SMM4H_2024_Task3_Training_1800.csv', nrows=1800, usecols=[1, 2, 3], engine='python')


In [ ]:
# Lematizar las columnas 'keyword' y 'text'
data['keyword'] = data['keyword'].apply(stem_text)
data['text'] = data['text'].apply(stem_text)

# Reemplazar los valores de la columna 'label' que no son 0 por 1
data['label'] = data['label'].apply(lambda x: 1 if x != 0 else x)


In [ ]:
# Partición de los datos en train, validation y test
train_data, temp_data = train_test_split(data, test_size=0.2, stratify=data['label'], random_state=42)
val_data, test_data = train_test_split(temp_data, test_size=0.5, stratify=temp_data['label'], random_state=42)

In [ ]:
# Preprocesamiento de texto y construcción del modelo
vectorizer = CountVectorizer()
model = MultinomialNB()

# Entrenar el modelo
X_train = vectorizer.fit_transform(train_data['text'])
y_train = train_data['label']
model.fit(X_train, y_train)

# Validación del modelo
X_val = vectorizer.transform(val_data['text'])
y_val = val_data['label']
predicted_labels = model.predict(X_val)


In [ ]:
# Evaluación del modelo
print("Resultados en datos de validación:")
print(classification_report(y_val, predicted_labels))

# Prueba del modelo
X_test = vectorizer.transform(test_data['text'])
y_test = test_data['label']
predicted_labels_test = model.predict(X_test)

# Evaluación del modelo en los datos de prueba
print("\nResultados en datos de prueba:")
print(classification_report(y_test, predicted_labels_test))

# Guardar el modelo en un archivo
joblib.dump(model, 'naive_bayes_model_multinomial_stem_count_nltk.pkl')
joblib.dump(vectorizer, 'vectorizer_multinomial_stem_count_nltk.pkl')

# Copiar el archivo del modelo a Google Drive
!cp naive_bayes_model_multinomial_stem_count_nltk.pkl '/content/drive/My Drive/'
!cp vectorizer_multinomial_stem_count_nltk.pkl '/content/drive/My Drive/'


Resultados en datos de validación:
              precision    recall  f1-score   support

           0       0.79      0.81      0.80       113
           1       0.67      0.63      0.65        67

    accuracy                           0.74       180
   macro avg       0.73      0.72      0.72       180
weighted avg       0.74      0.74      0.74       180


Resultados en datos de prueba:
              precision    recall  f1-score   support

           0       0.74      0.74      0.74       113
           1       0.56      0.55      0.56        67

    accuracy                           0.67       180
   macro avg       0.65      0.65      0.65       180
weighted avg       0.67      0.67      0.67       180





---

CON PROMEDIO DE PALABRAS KEYWORD/LABEL TFIDFVECTORIZER

In [ ]:
import pandas as pd
import numpy as np
import nltk
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
from nltk.stem import SnowballStemmer
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.model_selection import train_test_split
from sklearn.naive_bayes import MultinomialNB
from sklearn.metrics import classification_report

# Descargar el recurso 'stopwords'
nltk.download('stopwords')
nltk.download('punkt')

# Inicializar el stemmer en inglés
stemmer = SnowballStemmer('english')

# Función para realizar stemming en el texto
def stem_text(text):
    # Tokenizar el texto
    tokens = word_tokenize(text)
    # Realizar stemming en cada token y excluir las stopwords
    stemmed_words = [stemmer.stem(word) for word in tokens if word.lower() not in stopwords.words('english')]
    return ' '.join(stemmed_words)

# Cargar los datos
data = pd.read_csv('SMM4H_2024_Task3_Training_1800.csv', nrows=1800, usecols=[1, 2, 3], engine='python')

# Lematizar las columnas 'keyword' y 'text'
data['keyword'] = data['keyword'].apply(stem_text)
data['text'] = data['text'].apply(stem_text)

# Reemplazar los valores de la columna 'label' que no son 0 por 1
data['label'] = data['label'].apply(lambda x: 1 if x != 0 else x)

# Calcular el promedio de palabras en el texto para cada clase
data['word_count'] = data['text'].apply(lambda x: len(x.split()))
class_word_counts = data.groupby('label')['word_count'].mean()

# Normalizar los valores del promedio de palabras
max_word_count = class_word_counts.max()
min_word_count = class_word_counts.min()
normalized_class_word_counts = (class_word_counts - min_word_count) / (max_word_count - min_word_count)

# Añadir la característica normalizada como una columna adicional en los datos
train_data['normalized_word_count'] = train_data['label'].map(normalized_class_word_counts)
val_data['normalized_word_count'] = val_data['label'].map(normalized_class_word_counts)
test_data['normalized_word_count'] = test_data['label'].map(normalized_class_word_counts)

# Partición de los datos en train, validation y test
X_train = train_data[['keyword', 'normalized_word_count']]
X_val = val_data[['keyword', 'normalized_word_count']]
X_test = test_data[['keyword', 'normalized_word_count']]
y_train = train_data['label']
y_val = val_data['label']
y_test = test_data['label']

# Preprocesamiento de texto y construcción del modelo
vectorizer = TfidfVectorizer()
model = MultinomialNB()

# Entrenar el modelo
X_train_text = vectorizer.fit_transform(X_train['keyword'])
X_train_combined = np.hstack((X_train_text.toarray(), np.array(X_train['normalized_word_count']).reshape(-1, 1)))
model.fit(X_train_combined, y_train)

# Validación del modelo
X_val_text = vectorizer.transform(X_val['keyword'])
X_val_combined = np.hstack((X_val_text.toarray(), np.array(X_val['normalized_word_count']).reshape(-1, 1)))
predicted_labels = model.predict(X_val_combined)

# Evaluación del modelo
print("Resultados en datos de validación:")
print(classification_report(y_val, predicted_labels))

# Prueba del modelo
X_test_text = vectorizer.transform(X_test['keyword'])
X_test_combined = np.hstack((X_test_text.toarray(), np.array(X_test['normalized_word_count']).reshape(-1, 1)))
predicted_labels_test = model.predict(X_test_combined)

# Evaluación del modelo en los datos de prueba
print("\nResultados en datos de prueba:")
print(classification_report(y_test, predicted_labels_test))

# Guardar el modelo en un archivo
joblib.dump(model, 'naive_bayes_model_multinomial_prom_pal_keyword_label_tfidf.pkl')
joblib.dump(vectorizer, 'vectorizer_multinomial_prom_pal_keyword_label_tfidf.pkl')

# Copiar el archivo del modelo a Google Drive
!cp naive_bayes_model_multinomial_prom_pal_keyword_label_tfidf.pkl '/content/drive/My Drive/'
!cp vectorizer_multinomial_prom_pal_keyword_label_tfidf.pkl '/content/drive/My Drive/'


[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


Resultados en datos de validación:
              precision    recall  f1-score   support

           0       1.00      0.97      0.99       113
           1       0.96      1.00      0.98        67

    accuracy                           0.98       180
   macro avg       0.98      0.99      0.98       180
weighted avg       0.98      0.98      0.98       180


Resultados en datos de prueba:
              precision    recall  f1-score   support

           0       1.00      0.98      0.99       113
           1       0.97      1.00      0.99        67

    accuracy                           0.99       180
   macro avg       0.99      0.99      0.99       180
weighted avg       0.99      0.99      0.99       180





---

CON PROMEDIO DE PALABRAS KEYWORD/LABEL COUNTVECTORIZER



In [ ]:
import pandas as pd
import numpy as np
import nltk
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
from nltk.stem import SnowballStemmer
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.model_selection import train_test_split
from sklearn.naive_bayes import MultinomialNB
from sklearn.metrics import classification_report

# Descargar el recurso 'stopwords'
nltk.download('stopwords')
nltk.download('punkt')

# Inicializar el stemmer en inglés
stemmer = SnowballStemmer('english')

# Función para realizar stemming en el texto
def stem_text(text):
    # Tokenizar el texto
    tokens = word_tokenize(text)
    # Realizar stemming en cada token y excluir las stopwords
    stemmed_words = [stemmer.stem(word) for word in tokens if word.lower() not in stopwords.words('english')]
    return ' '.join(stemmed_words)

# Cargar los datos
data = pd.read_csv('SMM4H_2024_Task3_Training_1800.csv', nrows=1800, usecols=[1, 2, 3], engine='python')

# Lematizar las columnas 'keyword' y 'text'
data['keyword'] = data['keyword'].apply(stem_text)
data['text'] = data['text'].apply(stem_text)

# Reemplazar los valores de la columna 'label' que no son 0 por 1
data['label'] = data['label'].apply(lambda x: 1 if x != 0 else x)

# Calcular el promedio de palabras en el texto para cada clase
data['word_count'] = data['text'].apply(lambda x: len(x.split()))
class_word_counts = data.groupby('label')['word_count'].mean()

# Normalizar los valores del promedio de palabras
max_word_count = class_word_counts.max()
min_word_count = class_word_counts.min()
normalized_class_word_counts = (class_word_counts - min_word_count) / (max_word_count - min_word_count)

# Añadir la característica normalizada como una columna adicional en los datos
train_data['normalized_word_count'] = train_data['label'].map(normalized_class_word_counts)
val_data['normalized_word_count'] = val_data['label'].map(normalized_class_word_counts)
test_data['normalized_word_count'] = test_data['label'].map(normalized_class_word_counts)

# Partición de los datos en train, validation y test
X_train = train_data[['keyword', 'normalized_word_count']]
X_val = val_data[['keyword', 'normalized_word_count']]
X_test = test_data[['keyword', 'normalized_word_count']]
y_train = train_data['label']
y_val = val_data['label']
y_test = test_data['label']

# Preprocesamiento de texto y construcción del modelo
vectorizer = CountVectorizer()
model = MultinomialNB()

# Entrenar el modelo
X_train_text = vectorizer.fit_transform(X_train['keyword'])
X_train_combined = np.hstack((X_train_text.toarray(), np.array(X_train['normalized_word_count']).reshape(-1, 1)))
model.fit(X_train_combined, y_train)

# Validación del modelo
X_val_text = vectorizer.transform(X_val['keyword'])
X_val_combined = np.hstack((X_val_text.toarray(), np.array(X_val['normalized_word_count']).reshape(-1, 1)))
predicted_labels = model.predict(X_val_combined)

# Evaluación del modelo
print("Resultados en datos de validación:")
print(classification_report(y_val, predicted_labels))

# Prueba del modelo
X_test_text = vectorizer.transform(X_test['keyword'])
X_test_combined = np.hstack((X_test_text.toarray(), np.array(X_test['normalized_word_count']).reshape(-1, 1)))
predicted_labels_test = model.predict(X_test_combined)

# Evaluación del modelo en los datos de prueba
print("\nResultados en datos de prueba:")
print(classification_report(y_test, predicted_labels_test))

# Guardar el modelo en un archivo
joblib.dump(model, 'naive_bayes_model_multinomial_prom_pal_keyword_label_count.pkl')
joblib.dump(vectorizer, 'vectorizer_multinomial_prom_pal_keyword_label_count.pkl')

# Copiar el archivo del modelo a Google Drive
!cp naive_bayes_model_multinomial_prom_pal_keyword_label_count.pkl '/content/drive/My Drive/'
!cp vectorizer_multinomial_prom_pal_keyword_label_count.pkl '/content/drive/My Drive/'


[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


Resultados en datos de validación:
              precision    recall  f1-score   support

           0       1.00      0.96      0.98       113
           1       0.93      1.00      0.96        67

    accuracy                           0.97       180
   macro avg       0.97      0.98      0.97       180
weighted avg       0.97      0.97      0.97       180


Resultados en datos de prueba:
              precision    recall  f1-score   support

           0       1.00      0.97      0.99       113
           1       0.96      1.00      0.98        67

    accuracy                           0.98       180
   macro avg       0.98      0.99      0.98       180
weighted avg       0.98      0.98      0.98       180





---

CON PROMEDIO DE PALABRAS TEXT/LABEL COUNTVECTORIZER



In [ ]:
import pandas as pd
import numpy as np
import nltk
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
from nltk.stem import SnowballStemmer
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.model_selection import train_test_split
from sklearn.naive_bayes import MultinomialNB
from sklearn.metrics import classification_report

# Descargar el recurso 'stopwords'
nltk.download('stopwords')
nltk.download('punkt')

# Inicializar el stemmer en inglés
stemmer = SnowballStemmer('english')

# Función para realizar stemming en el texto
def stem_text(text):
    # Tokenizar el texto
    tokens = word_tokenize(text)
    # Realizar stemming en cada token y excluir las stopwords
    stemmed_words = [stemmer.stem(word) for word in tokens if word.lower() not in stopwords.words('english')]
    return ' '.join(stemmed_words)

# Cargar los datos
data = pd.read_csv('SMM4H_2024_Task3_Training_1800.csv', nrows=1800, usecols=[1, 2, 3], engine='python')

# Lematizar las columnas 'keyword' y 'text'
data['keyword'] = data['keyword'].apply(stem_text)
data['text'] = data['text'].apply(stem_text)

# Reemplazar los valores de la columna 'label' que no son 0 por 1
data['label'] = data['label'].apply(lambda x: 1 if x != 0 else x)

# Calcular el promedio de palabras en el texto para cada clase
data['word_count'] = data['text'].apply(lambda x: len(x.split()))
class_word_counts = data.groupby('label')['word_count'].mean()

# Normalizar los valores del promedio de palabras
max_word_count = class_word_counts.max()
min_word_count = class_word_counts.min()
normalized_class_word_counts = (class_word_counts - min_word_count) / (max_word_count - min_word_count)

# Añadir la característica normalizada como una columna adicional en los datos
train_data['normalized_word_count'] = train_data['label'].map(normalized_class_word_counts)
val_data['normalized_word_count'] = val_data['label'].map(normalized_class_word_counts)
test_data['normalized_word_count'] = test_data['label'].map(normalized_class_word_counts)

# Partición de los datos en train, validation y test
X_train = train_data[['text', 'normalized_word_count']]
X_val = val_data[['text', 'normalized_word_count']]
X_test = test_data[['text', 'normalized_word_count']]
y_train = train_data['label']
y_val = val_data['label']
y_test = test_data['label']

# Preprocesamiento de texto y construcción del modelo
vectorizer = CountVectorizer()
model = MultinomialNB()

# Entrenar el modelo
X_train_text = vectorizer.fit_transform(X_train['text'])
X_train_combined = np.hstack((X_train_text.toarray(), np.array(X_train['normalized_word_count']).reshape(-1, 1)))
model.fit(X_train_combined, y_train)

# Validación del modelo
X_val_text = vectorizer.transform(X_val['text'])
X_val_combined = np.hstack((X_val_text.toarray(), np.array(X_val['normalized_word_count']).reshape(-1, 1)))
predicted_labels = model.predict(X_val_combined)

# Evaluación del modelo
print("Resultados en datos de validación:")
print(classification_report(y_val, predicted_labels))

# Prueba del modelo
X_test_text = vectorizer.transform(X_test['text'])
X_test_combined = np.hstack((X_test_text.toarray(), np.array(X_test['normalized_word_count']).reshape(-1, 1)))
predicted_labels_test = model.predict(X_test_combined)

# Evaluación del modelo en los datos de prueba
print("\nResultados en datos de prueba:")
print(classification_report(y_test, predicted_labels_test))

# Guardar el modelo en un archivo
joblib.dump(model, 'naive_bayes_model_multinomial_prom_pal_text_label_count.pkl')
joblib.dump(vectorizer, 'vectorizer_multinomial_prom_pal_text_label_count.pkl')

# Copiar el archivo del modelo a Google Drive
!cp naive_bayes_model_multinomial_prom_pal_text_label_count.pkl '/content/drive/My Drive/'
!cp vectorizer_multinomial_prom_pal_text_label_count.pkl '/content/drive/My Drive/'


[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


Resultados en datos de validación:
              precision    recall  f1-score   support

           0       0.94      0.86      0.90       113
           1       0.79      0.91      0.85        67

    accuracy                           0.88       180
   macro avg       0.87      0.88      0.87       180
weighted avg       0.89      0.88      0.88       180


Resultados en datos de prueba:
              precision    recall  f1-score   support

           0       0.90      0.80      0.85       113
           1       0.71      0.85      0.78        67

    accuracy                           0.82       180
   macro avg       0.81      0.82      0.81       180
weighted avg       0.83      0.82      0.82       180





---



CON PROMEDIO DE PALABRAS TEXT/LABEL TFIDFVECTORIZER


In [ ]:
import pandas as pd
import numpy as np
import nltk
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
from nltk.stem import SnowballStemmer
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.model_selection import train_test_split
from sklearn.naive_bayes import MultinomialNB
from sklearn.metrics import classification_report

# Descargar el recurso 'stopwords'
nltk.download('stopwords')
nltk.download('punkt')

# Inicializar el stemmer en inglés
stemmer = SnowballStemmer('english')

# Función para realizar stemming en el texto
def stem_text(text):
    # Tokenizar el texto
    tokens = word_tokenize(text)
    # Realizar stemming en cada token y excluir las stopwords
    stemmed_words = [stemmer.stem(word) for word in tokens if word.lower() not in stopwords.words('english')]
    return ' '.join(stemmed_words)

# Cargar los datos
data = pd.read_csv('SMM4H_2024_Task3_Training_1800.csv', nrows=1800, usecols=[1, 2, 3], engine='python')

# Lematizar las columnas 'keyword' y 'text'
data['keyword'] = data['keyword'].apply(stem_text)
data['text'] = data['text'].apply(stem_text)

# Reemplazar los valores de la columna 'label' que no son 0 por 1
data['label'] = data['label'].apply(lambda x: 1 if x != 0 else x)

# Calcular el promedio de palabras en el texto para cada clase
data['word_count'] = data['text'].apply(lambda x: len(x.split()))
class_word_counts = data.groupby('label')['word_count'].mean()
print(class_word_counts)

# Normalizar los valores del promedio de palabras
max_word_count = class_word_counts.max()
min_word_count = class_word_counts.min()
normalized_class_word_counts = (class_word_counts - min_word_count) / (max_word_count - min_word_count)

# Añadir la característica normalizada como una columna adicional en los datos
train_data['normalized_word_count'] = train_data['label'].map(normalized_class_word_counts)
val_data['normalized_word_count'] = val_data['label'].map(normalized_class_word_counts)
test_data['normalized_word_count'] = test_data['label'].map(normalized_class_word_counts)

# Partición de los datos en train, validation y test
X_train = train_data[['text', 'normalized_word_count']]
X_val = val_data[['text', 'normalized_word_count']]
X_test = test_data[['text', 'normalized_word_count']]
y_train = train_data['label']
y_val = val_data['label']
y_test = test_data['label']

print('test_data',X_test)

# Preprocesamiento de texto y construcción del modelo
vectorizer = TfidfVectorizer()
model = MultinomialNB()

# Entrenar el modelo
X_train_text = vectorizer.fit_transform(X_train['text'])
X_train_combined = np.hstack((X_train_text.toarray(), np.array(X_train['normalized_word_count']).reshape(-1, 1)))
model.fit(X_train_combined, y_train)

# Validación del modelo
X_val_text = vectorizer.transform(X_val['text'])
X_val_combined = np.hstack((X_val_text.toarray(), np.array(X_val['normalized_word_count']).reshape(-1, 1)))
predicted_labels = model.predict(X_val_combined)

# Evaluación del modelo
print("Resultados en datos de validación:")
print(classification_report(y_val, predicted_labels))

# Prueba del modelo
X_test_text = vectorizer.transform(X_test['text'])
X_test_combined = np.hstack((X_test_text.toarray(), np.array(X_test['normalized_word_count']).reshape(-1, 1)))
predicted_labels_test = model.predict(X_test_combined)

# Evaluación del modelo en los datos de prueba
print("\nResultados en datos de prueba:")
print(classification_report(y_test, predicted_labels_test))

# Guardar el modelo en un archivo
joblib.dump(model, 'naive_bayes_model_multinomial_prom_pal_text_label_tfidf.pkl')
joblib.dump(vectorizer, 'vectorizer_multinomial_prom_pal_text_label_tfidf.pkl')

# Copiar el archivo del modelo a Google Drive
!cp naive_bayes_model_multinomial_prom_pal_text_label_tfidf.pkl '/content/drive/My Drive/'
!cp vectorizer_multinomial_prom_pal_text_label_tfidf.pkl '/content/drive/My Drive/'


[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


label
0    111.706454
1    150.692078
Name: word_count, dtype: float64
test_data                                                    text  normalized_word_count
1573  feel ya buddi , spent day loung around home , ...                    1.0
460   sure tell regard possibl swim problem futur , ...                    0.0
315   although subject frustrat , n't let beauti thi...                    0.0
1659  struggl get outsid . start thing : 'm 21 year ...                    1.0
1179  recent went run / walk . took like week actual...                    1.0
...                                                 ...                    ...
161   sever anxiety/depress . 'm 20 year old dude di...                    0.0
654   anxieti roof ! . past week two get rid one dog...                    0.0
957   n't realli tast music . younger probabl tast w...                    0.0
744                         ’ outsid though . ’ peopl .                    0.0
619   pretti much , 's exact get thing done make c

In [ ]:
# Transformar los datos de texto de todos los datos utilizando el mismo vectorizador
X_all_data_text = vectorizer.transform(data['text'])

# Calcular el promedio de palabras en el texto para cada clase
data['word_count'] = data['text'].apply(lambda x: len(x.split()))
class_word_counts = data.groupby('label')['word_count'].mean()

# Normalizar los valores del promedio de palabras
max_word_count = class_word_counts.max()
min_word_count = class_word_counts.min()
normalized_class_word_counts = (class_word_counts - min_word_count) / (max_word_count - min_word_count)

# Añadir la característica normalizada como una columna adicional en los datos
data['normalized_word_count'] = data['label'].map(normalized_class_word_counts)

# Transformar los datos de texto de todos los datos utilizando el mismo vectorizador
X_all_data_text = vectorizer.transform(data['text'])

# Combinar las características de texto y la característica normalizada
X_all_data_combined = np.hstack((X_all_data_text.toarray(), np.array(data['normalized_word_count']).reshape(-1, 1)))

# Predecir las etiquetas para todos los datos
predicted_labels_all_data = model.predict(X_all_data_combined)

# Evaluación del modelo en todos los datos
print("\nResultados en todos los datos:")
print(classification_report(data['label'], predicted_labels_all_data))



Resultados en todos los datos:
              precision    recall  f1-score   support

           0       1.00      1.00      1.00      1131
           1       1.00      1.00      1.00       669

    accuracy                           1.00      1800
   macro avg       1.00      1.00      1.00      1800
weighted avg       1.00      1.00      1.00      1800



In [ ]:
# Cargar el archivo de prueba sin las etiquetas de clase
test_data = pd.read_csv('SMM4H_2024_Task3_Validation_600_release.csv', nrows=600, usecols=[0, 1, 2], engine='python')

# Lematizar las columnas 'keyword' y 'text'
test_data['keyword'] = test_data['keyword'].apply(stem_text)
test_data['text'] = test_data['text'].apply(stem_text)

# Calcular el promedio de palabras en el texto para cada clase
test_data['word_count'] = test_data['text'].apply(lambda x: len(x.split()))

print(test_data)

# Normalizar los valores del promedio de palabras
test_data['normalized_word_count'] = (test_data['word_count'] - min_word_count) / (max_word_count - min_word_count)

# Transformar los datos de texto utilizando el mismo vectorizador que se ajustó en los datos de entrenamiento
X_test_text = vectorizer.transform(test_data['text'])

# Combinar las características de texto con la característica normalizada
X_test_combined = np.hstack((X_test_text.toarray(), np.array(test_data['normalized_word_count']).reshape(-1, 1)))

# Predecir las etiquetas para los datos de prueba
predicted_labels_test = model.predict(X_test_combined)

# Mostrar los resultados
print("\nResultados en los nuevos datos:")
for text, predicted_label in zip(test_data['text'], predicted_labels_test):
    print(f"Texto: {text}\nClase predicha: {predicted_label}\n")

# Contar la frecuencia de cada clase predicha
class_counts = np.bincount(predicted_labels_test)

# Mostrar el conteo de cada clase
for label, count in enumerate(class_counts):
    print(f"Clase {label}: {count} instancias")
